In [1]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2Config, DataCollatorWithPadding
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
model = GPT2LMHeadModel.from_pretrained("output/checkpoint-13500/").to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [42]:
sentence = '깊은 밤의 불안한 별빛'
input_ids = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='pt').to(device)

with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        temperature=0.2, # 생성 다양성 조절
        max_new_tokens=64, # 생성되는 문장의 최대 길이
        top_k=25, # 높은 확률을 가진 top-k 토큰만 고려
        top_p=0.95, # 누적 확률이 p를 초과하는 토큰은 제외
        repetition_penalty=1.2, # 반복을 줄이는 패널티
        do_sample=True, # 샘플링 기반 생성 활성화
        num_return_sequences=2, # 생성할 시퀀스의 수
    )

text_list = []
for output_text in output:
    text_list.append(tokenizer.decode(output_text.tolist(), skip_special_tokens=True))

# generated_sequence = output[0].tolist()
# text = tokenizer.decode(generated_sequence, skip_special_tokens=True)

In [43]:
for i in range(len(text_list)):
    print(f"----------{i+1}번째 시----------")
    print(text_list[i])

----------1번째 시----------
깊은 밤의 불안한 별빛에
그 떨림이 멈추어 있는 
별들의 마음에도
이렇게 찬란하도록 빛나는 시간이 있음을 알게 됩니다
그래서 아름다운 날에요
고운 새들이 가져온
하얀 눈송이가 들려주는
꿈의 이야기들
그리고 달팽이의 하얀 눈이 내리는 곳마다
아름다움의 깊이
----------2번째 시----------
깊은 밤의 불안한 별빛에
어둠이 스며들면
그렇게 꿈은 사라지고 
아름다움의 시간 속에 있는
고운 새들의 아침과 저녁놀의 시작을 알려줍니다
그래서 아름다운 날에요
별들이 보내온
하얀 눈송이의 편지 안에
꿈으로 가는 길에는
마음이 닿는


In [ ]:
sentence = '돈'
toked = tokenizer("</s>" + sentence)
temp = []
cnt = 0
while True:
  input_ids = torch.tensor(toked["input_ids"]).unsqueeze(0).to(device)
  pred = model(input_ids)[0]

  gen = tokenizer.decode(torch.argmax(pred, axis=-1).squeeze().tolist())
  print(gen)
  print("선택한 단어 : ", gen[-1])
  gen = gen[-1]
  cnt += 1

  if cnt == 50:
    break

  if '' == gen:
    break
  sentence += gen.replace('▁', ' ')
  toked = tokenizer(sentence)

print(sentence)